In [1]:
import os, sys, time
import numpy as np
cwd = os.getcwd()
parent = os.path.join(cwd, os.path.join(os.path.dirname("__file__")))
sys.path.append(os.path.join(parent, 'chofer_torchex'))
sys.path.append(os.path.join(parent, 'tda-toolkit'))

from src.animal.generate_dgm_provider import generate_dgm_provider
from src.animal.experiments import experiment
from src.sharedCode.data_downloader import download_provider, download_raw_data
from src.sharedCode.gui import ask_user_for_provider_or_data_set_download
provider_path = os.path.join(os.path.dirname("__file__"), 'data/dgm_provider/npht_animal_32dirs_from_raw.h5')
raw_data_path = os.path.join(os.path.dirname("__file__"), 'data/raw_data/animal')

/notebooks/DockerShared/Pytorch3/TDA_hofer_2017/tda-toolkit/pershombox/_software_backends/resource_handler.py:91: UserWarning: The following backends are not properly configured
dipha
hera_wasserstein_dist
perseus
Using stuff dependent on those backends will cause runtime errors.
You can get all errors by calling pershombox.get_backend_cfg_errors().

  warnings.warn(error_text, UserWarning)


In [2]:
# download_raw_data("animal")
generate_dgm_provider(raw_data_path, provider_path, 32)


STARTdone 0/2000


Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessi

KeyboardInterrupt: 

In [4]:
from skimage import io
import scipy.misc
import imageio
import matplotlib.pyplot as plt
%matplotlib inline

sample_file_path = "data/raw_data/animal/bird/bird1.tif"
# same 
img = io.imread(sample_file_path, as_gray=True).astype(np.float32)
img1 = scipy.misc.imread(sample_file_path, flatten=True)
img2 = imageio.imread(sample_file_path)
plt.imshow(img)

In [9]:
import os
import multiprocessing
from skimage import io
import scipy.ndimage

from src.sharedCode.fileSys import Folder
from src.sharedCode.gui import SimpleProgressCounter
from src.sharedCode.provider import Provider
from src.sharedCode.generate_dgm_provider_shapes import *

In [ ]:
data_path, output_file_path, number_of_directions = \
raw_data_path, provider_path, 32

if not os.path.exists(os.path.dirname(output_file_path)):
    print(os.path.dirname(output_file_path), 'does not exist.')
src_folder = Folder(data_path)
class_folders = src_folder.folders()
n = sum([len(cf.files(name_pred=lambda n: n != 'Thumbs.db')) for cf in class_folders])
progress = SimpleProgressCounter(n)
progress.display()
views = {}
for i in range(0, number_of_directions):
    views['dim_0_dir_{}'.format(i)] = {}
    views['dim_1_dir_{}'.format(i)] = {}
job_args = []
for class_folder in class_folders:
    for view in views.values():
        view[class_folder.name] = {}
    for sample_file in class_folder.files(name_pred=lambda name: name != 'Thumbs.db'):
        args = {'file_path': sample_file.path,
                'label': class_folder.name,
                'sample_id': sample_file.name,
                'number_of_directions': number_of_directions}
        job_args.append(args)
n_cores = int(multiprocessing.cpu_count()*0.5)


def job(args):
    import warnings
    warnings.filterwarnings('error')#TODO change this ... those errors should not occour
    sample_file_path = args['file_path']
    label = args['label']
    sample_id = args['sample_id']
    number_of_directions = args['number_of_directions']
    return_value = {'label': label, 'sample_id': sample_id, 'dgms': {}}
    img = imageio.imread(sample_file_path)
    img = reduce_to_largest_connected_component(img)
    try:
        npht = get_npht(img, number_of_directions)
    except Exception as ex:
        return_value['error'] = ex
    else:
        dgms_dim_0 = [x[0] for x in npht]
        dgms_dim_1 = [x[1] for x in npht]
        dgms_dim_0 = [threhold_dgm(dgm) for dgm in dgms_dim_0]
        dgms_dim_1 = [threhold_dgm(dgm) for dgm in dgms_dim_1]
        for dir_i, dgm_0, dgm_1 in zip(range(number_of_directions), dgms_dim_0, dgms_dim_1):
            if len(dgm_0) == 0:
                return_value['error'] = 'Degenerate diagram detected.'
                break
            return_value['dgms']['dim_0_dir_{}'.format(dir_i)] = dgm_0
            return_value['dgms']['dim_1_dir_{}'.format(dir_i)] = dgm_1
    return return_value

In [ ]:
with multiprocessing.Pool(n_cores) as pool:
    errors = []
    for i, result in enumerate(pool.imap_unordered(job, job_args)):
        label = result['label']
        sample_id = result['sample_id']
        if 'error' in result:
            errors.append((sample_id, result['error']))
        else:
            for view_id, dgm in result['dgms'].items():
                views[view_id][label][sample_id] = dgm
        progress.trigger_progress()
print('Finished calculation ... writing provider (this may also take some time ;) 

In [ ]:
prv = Provider()
for key, view_data in views.items():
    prv.add_view(key, view_data)
meta = {'number_of_directions': number_of_directions}
prv.add_meta_data(meta)
prv.dump_as_h5(output_file_path)
if len(errors) > 0:
    with open('animal_dgm_creation_errors.txt', 'w') as f:
        for k, v in errors:
            f.write(k)
            f.write(str(v))

# check training code

In [15]:
import torch
from src.animal.experiments import _parameters, _data_setup
from src.animal.experiments import MyModel, _create_trainer

In [17]:
params = _parameters()
data_path = provider_path
params['data_path'] = data_path
if torch.cuda.is_available(): params['cuda'] = True
print('Data setup...')
data_train, data_test, subscripted_views = _data_setup(params)

Loading provider...
Create data loader...


In [25]:
dataiter = iter(data_train)
d1 = dataiter.next()

In [30]:
len(d1)

2

In [33]:
d1[0]['dim_0_dir_0'][0]

tensor([[0.1055, 0.9428]])

In [36]:
print('Create model...')
model = MyModel(subscripted_views)

Create model...


/notebooks/DockerShared/Pytorch3/TDA_hofer_2017/chofer_torchex/chofer_torchex/nn/slayer.py:338: FutureWarning: Renaming in progress. In future use SLayerExponential.
  warnings.warn("Renaming in progress. In future use SLayerExponential.", FutureWarning)


In [41]:
model

MyModel(
  (pht_sl): SLayerPHT(
    (sl_0): SLayerExponential (... -> 75 )
    (sl_1): SLayerExponential (... -> 75 )
    (sl_2): SLayerExponential (... -> 75 )
    (sl_3): SLayerExponential (... -> 75 )
    (sl_4): SLayerExponential (... -> 75 )
    (sl_5): SLayerExponential (... -> 75 )
    (sl_6): SLayerExponential (... -> 75 )
    (sl_7): SLayerExponential (... -> 75 )
    (sl_8): SLayerExponential (... -> 75 )
    (sl_9): SLayerExponential (... -> 75 )
    (sl_10): SLayerExponential (... -> 75 )
    (sl_11): SLayerExponential (... -> 75 )
    (sl_12): SLayerExponential (... -> 75 )
    (sl_13): SLayerExponential (... -> 75 )
    (sl_14): SLayerExponential (... -> 75 )
    (sl_15): SLayerExponential (... -> 75 )
    (sl_16): SLayerExponential (... -> 75 )
    (sl_17): SLayerExponential (... -> 75 )
    (sl_18): SLayerExponential (... -> 75 )
    (sl_19): SLayerExponential (... -> 75 )
    (sl_20): SLayerExponential (... -> 75 )
    (sl_21): SLayerExponential (... -> 75 )
    (sl_22

In [44]:
print('Setup trainer...')
trainer = _create_trainer(model, params, data_train, data_test)

In [50]:
trainer??

In [ ]:
print('Starting...')
trainer.run()
last_10_accuracies = list(trainer.prediction_monitor.accuracies.values())[-10:]
mean = np.mean(last_10_accuracies)